# Trabalho 3

### Dupla

Andreina Maria Mendes da Silva - 485306  
Augusto César Araújo de Oliveira - 508991

## Descrição do Trabalho

- Preparação dos dados:
  - Baixar e explorar o conjunto de dados escolhido
  - Dividir o conjunto de dados em treino, validação e teste
  - Realizar pré-processamento adequado ao tipo de dados
- Criação do modelo:
  - Projetar uma arquitetura de rede neural apropriada para o tipo de dado escolhido.
- Treinamento:
  - Implementar o loop de treinamento
  - Escolher otimizador e função de perda adequados
  - Treinar o modelo por várias épocas
- Avaliação:
  - Avaliar o desempenho do modelo no conjunto de teste
  - Calcular métricas relevantes.
- Visualização e interpretação:
  - Criar visualizações apropriadas para o tipo de dados
  - Interpretar algumas previsões do modelo
- Conclusões sobre o trabalho:
  - Explicar o processo, as decisões tomadas, as dificuldades, as lições aprendidas e os resultados.

# Importação das bibliotecas

In [3]:
!pip install torch
!pip install torchvision

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.3 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 3.9 MB/s eta 0:00:00


In [3]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import models, transforms
from torchvision.models import resnet50, ResNet50_Weights

import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# Importação dos Dados

## Pastas utilizadas

In [4]:
data_dir = "archive"
train = 'train'
val = 'val'
test = 'test'

## Função para renomear as imagens e mudar a pasta delas

In [5]:
def transform_cls(root):
    for cls in os.listdir(root):
        for i, img in enumerate(os.listdir(os.path.join(root, cls))):
            new_name = f"{cls.lower()}_{i}.jpg"  # Nome único baseado no índice
            os.rename(os.path.join(root, cls, img), os.path.join(root, new_name))

### Chamada de função para as imagens de treino

In [6]:
transform_cls(os.path.join(data_dir, train))

### Chamada de função para as imagens de teste

In [7]:
transform_cls(os.path.join(data_dir, test))

## Função para criar o conjunto de validação a partir do treino

In [8]:
def make_validation_data(data_dir, train):
    root = os.path.join(data_dir, train)
    val_img_path = []

    for img in os.listdir(root):
        label = img.split('.')[0].split('_')[0]
        if len(val_img_path) < 10000 and label == 'fake':
            val_img_path.append(os.path.join(root, img))
        elif len(val_img_path) < 20000 and label == 'real':
            val_img_path.append(os.path.join(root, img))

    # Diretório de validação
    val_dir = os.path.join(data_dir, 'val')  
    if not os.path.exists(val_dir):
        os.mkdir(val_dir)  

    # Mover as imagens para o diretório de validação
    for img_path in val_img_path:
        # Extrai o nome completo da imagem, não apenas a extensão
        img_name = os.path.basename(img_path)
        os.rename(img_path, os.path.join(val_dir, img_name))  


### Chamada de função

In [9]:
make_validation_data(data_dir, train)

# Preparação dos Dados

## Classe das Imagens do Dataset

Sobre os labels:
- FAKE = 0
- REAL = 1

In [10]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir: str, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        
        # Filtrar para incluir apenas arquivos e ignorar diretórios
        self.image_paths = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_paths[idx])
        image = Image.open(img_path)
        
        # Extrair o rótulo do nome da imagem
        label_str = self.image_paths[idx].split('_')[0]
        
        if label_str == 'fake':
            label = torch.tensor(0)
        else:
            label = torch.tensor(1)

        if self.transform:
            image = self.transform(image)

        return image, label

## Transformações

- Reajuste do tamanho das imagens
- Conversão para Tensor

In [11]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.PILToTensor(),
])

## Inicia o dataset de imagens com a classe, aplicando as transformações

In [12]:
train_dataset = CustomImageDataset(os.path.join(data_dir, train), transform)
val_dataset = CustomImageDataset(os.path.join(data_dir, val), transform)
test_dataset = CustomImageDataset(os.path.join(data_dir, test), transform)

## Cria o dataloader para cada um dos conjuntos (treino, teste, validação)

In [13]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

## Calcula a próxima iteração para o treino

In [14]:
train_images, train_labels = next(iter(train_dataloader))

In [15]:
print(train_images.shape)
print(train_labels.shape)

torch.Size([64, 3, 32, 32])
torch.Size([64])


## Calcula a próxima iteração para a validação

In [16]:
val_images, val_labels = next(iter(val_dataloader))
print(val_images.shape)
print(val_labels.shape)

torch.Size([64, 3, 32, 32])
torch.Size([64])


## Calcula a próxima iteração para o teste

In [17]:
test_images, test_labels = next(iter(test_dataloader))
print(test_images.shape)
print(test_labels.shape)

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [62]:
classes = [cls.lower() for cls in os.listdir(os.path.join(data_dir, train))]
print(classes)

['fake', 'fake_18999.jpg', 'fake_19.jpg', 'fake_190.jpg', 'fake_1900.jpg', 'fake_19000.jpg', 'fake_19001.jpg', 'fake_19002.jpg', 'fake_19003.jpg', 'fake_19004.jpg', 'fake_19005.jpg', 'fake_19006.jpg', 'fake_19007.jpg', 'fake_19008.jpg', 'fake_19009.jpg', 'fake_1901.jpg', 'fake_19010.jpg', 'fake_19011.jpg', 'fake_19012.jpg', 'fake_19013.jpg', 'fake_19014.jpg', 'fake_19015.jpg', 'fake_19016.jpg', 'fake_19017.jpg', 'fake_19018.jpg', 'fake_19019.jpg', 'fake_1902.jpg', 'fake_19020.jpg', 'fake_19021.jpg', 'fake_19022.jpg', 'fake_19023.jpg', 'fake_19024.jpg', 'fake_19025.jpg', 'fake_19026.jpg', 'fake_19027.jpg', 'fake_19028.jpg', 'fake_19029.jpg', 'fake_1903.jpg', 'fake_19030.jpg', 'fake_19031.jpg', 'fake_19032.jpg', 'fake_19033.jpg', 'fake_19034.jpg', 'fake_19035.jpg', 'fake_19036.jpg', 'fake_19037.jpg', 'fake_19038.jpg', 'fake_19039.jpg', 'fake_1904.jpg', 'fake_19040.jpg', 'fake_19041.jpg', 'fake_19042.jpg', 'fake_19043.jpg', 'fake_19044.jpg', 'fake_19045.jpg', 'fake_19046.jpg', 'fake_19047

# Inferência Inicial

## Define os pesos default para o modelo (pré-treinado)

In [66]:
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

preprocess = weights.transforms()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\augxu/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100.0%


## Determina o número de classes e cria as camadas fully-conected

In [67]:
num_classes = 2 
model.fc = nn.Linear(model.fc.in_features, num_classes)

## Faz usar GPU, senão CPU

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

## Define as transformações padrões para o modelo ResNet

- Redimensionamento das imagens ocorre por interpolação

In [2]:
# Definindo as transformações (Redimensionamento e conversão para tensor)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Tamanho esperado pelo ResNet50
    transforms.PILToTensor(),  # Converte a imagem para tensor não normalizado
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalização padrão do ResNet50
])

NameError: name 'transforms' is not defined

## Customiza as imagens de treino novamente para as novas transformações e carrega o dataloader de teste

In [76]:
test_dataset = CustomImageDataset(os.path.join(data_dir, test), transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

## Cria uma função para rodar a inferência inicial

In [77]:
def run_inference(model, dataloader, device):
    model.eval()  # Garantir que o modelo esteja em modo de avaliação
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Não queremos calcular gradientes durante a inferência
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Fazer a inferência
            outputs = model(images)

            # Pegar as previsões (o índice com a maior probabilidade)
            _, preds = torch.max(outputs, 1)

            # Guardar previsões e rótulos
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    return torch.cat(all_preds), torch.cat(all_labels)

### Execução da inferência inicial

In [78]:
test_preds_inf, test_labels_inf = run_inference(model, test_dataloader, device)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\augxu\\OneDrive\\Documentos\\Aprendizado de máquina\\cifake\\archive\\test\\FAKE'

### Mostrar o tamanho das previsões e rótulos para validação

In [ ]:
print(f'Previsões para {len(test_preds_inf)} imagens.')
print(f'Primeiras previsões: {test_preds_inf[:10]}')
print(f'Primeiros rótulos: {test_labels_inf[:10]}')

### Mostrar a acurácia para o conjunto de teste sem reajuste dos pesos

In [ ]:
accuracy_inf = (test_preds_inf == test_labels_inf).sum().item() / len(test_labels_inf)
print(f'Acurácia inicial no conjunto de treino: {accuracy_inf * 100:.2f}%')

# Fine Tuning